In [2]:
%pip install matplotlib
%pip install seaborn
%pip install hvplot
%pip install xgboost
%pip install scikit-learn
%pip install tensorflow
%pip install pandas
%pip install psycopg2



Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pickle
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

from helper.postgres import create_dataframe_for_table

In [4]:
doctors_df = create_dataframe_for_table('doctors')
print(doctors_df.columns)
doctors_df = doctors_df.rename(columns={'id': 'doctor_id'})
doctors_df.head()

Index(['id', 'name', 'rating', 'num_ratings', 'phone', 'speciality',
       'practice_name', 'street_address', 'city', 'state', 'postal_code'],
      dtype='object')


,doctor_id,name,rating,num_ratings,phone,speciality,practice_name,street_address,city,state,postal_code
0,1,"Dr. Frank Shea, MD",2.4,25,297.601.3742,Cardiologist,Williamsview Health - Schroeder,8096 Day Unions Apt. 527,Christopherhaven,New Hampshire,84644
1,2,"Dr. Wayne Johnson, MD",3.1,55,514.821.5740,Cardiologist,Nashton Health - Hicks,0700 Wilson Prairie,Port Nicholeton,Arizona,06132
2,3,"Dr. Todd Bauer, MD",1.0,64,474-283-7327x0667,Cardiologist,West Jerry Health - Powell,9219 Harper Lane,Santoschester,New Jersey,05754
3,4,"Dr. Paul Ellis, MD",2.3,58,+1-738-434-8951,Cardiologist,Sarahport Health - Bailey,1543 Joseph Fort Apt. 241,Hartborough,Nebraska,96703
4,5,"Dr. Melissa Beck, MD",2.5,47,7257250376,Cardiologist,Hamptonville Health - Cole,6953 Melissa Ford Suite 930,Harmonhaven,Nevada,89332


In [5]:
doctor_reviews_df = create_dataframe_for_table('doctor_reviews')
print(doctor_reviews_df.columns)
doctor_reviews_df.head()

Index(['doctor_id', 'rating', 'date_published', 'user_id', 'id'], dtype='object')


,doctor_id,rating,date_published,user_id,id
0,1,4,"Nov 22, 2024",92,1
1,1,3,"Sep 18, 2024",184,2
2,2,1,"Oct 10, 2024",259,3
3,3,1,"Oct 17, 2024",105,4
4,3,1,"Aug 07, 2024",85,5


In [6]:
user_df = create_dataframe_for_table('users')
print(user_df.columns)
user_df = user_df.rename(columns={'id': 'user_id'})
user_df.head()

Index(['id', 'name', 'email', 'street_address', 'city', 'state', 'postal_code',
       'password'],
      dtype='object')


,user_id,name,email,street_address,city,state,postal_code,password
0,1,Michael Skinner,donald21@example.org,2259 Elizabeth Tunnel,East Matthewstad,Wyoming,45744,gAAAAABnUfpE2-kijiaDzaEO-W8WbrS0xqclpf_QPCaKdB...
1,2,Sharon Lewis,diane45@example.org,294 Wayne Center,Clarkburgh,Indiana,15426,gAAAAABnUfpE5n_pbxZEzJ4gX6RHNCcXFf-mJTIyH2Rxbn...
2,3,John Phillips,johnsonamy@example.com,018 Lisa Circles,Jonesbury,Missouri,18011,gAAAAABnUfpErnPOqUq-AkcE-LV__Z2mEGTwNQ0eX1QvsO...
3,4,Christina Watkins,larrygray@example.net,12544 Landry Branch,New Jill,Illinois,77535,gAAAAABnUfpEVyYpJdwiwzOmNmxKn7tGtb099b8xLe8coS...
4,5,Matthew Murillo,peterthompson@example.net,9511 Sharon Motorway Apt. 413,Lake Karenberg,Florida,46497,gAAAAABnUfpEvwSUajW97M3A8nRRWIswD_vu0rl7BPSGBa...


In [7]:
user_report_df = create_dataframe_for_table('user_report')
print(user_report_df.columns)
user_report_df.head()

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target', 'user_id', 'id'],
      dtype='object')


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,user_id,id
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1,2,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1,3,3
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1,4,4
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1,5,5


In [8]:
# merge user_report score, user - location, doctors - id,location, doctor_review

In [9]:
# user_report_merged = pd.merge(user_df, user_report_df, on='user_id')
# user_report_merged = user_report_merged[['score', 'user_id', 'street_address', 'city', 'state', 'postal_code']]
final_df = pd.merge(user_report_df, doctor_reviews_df, on='user_id')
final_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,user_id,id_x,doctor_id,rating,date_published,id_y
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,1,1,16,1,"May 20, 2024",79
1,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,1,1,30,5,"Dec 25, 2023",175
2,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,1,1,98,1,"Jul 17, 2024",515
3,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,1,1,129,1,"Mar 04, 2024",669
4,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1,2,2,7,2,"Jan 16, 2024",32


In [10]:
final_df = final_df.drop(['user_id', 'id_x', 'date_published', 'id_y', 'target'], axis=1)
final_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,doctor_id,rating
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,16,1
1,63,1,3,145,233,1,0,150,0,2.3,0,0,1,30,5
2,63,1,3,145,233,1,0,150,0,2.3,0,0,1,98,1
3,63,1,3,145,233,1,0,150,0,2.3,0,0,1,129,1
4,37,1,2,130,250,0,1,187,0,3.5,0,0,2,7,2


In [11]:
X = final_df.drop(['doctor_id'], axis=1)
y = final_df['doctor_id']

In [12]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [14]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [15]:
y_pred = knn.predict(X_test)

In [16]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.006369426751592357


In [17]:
# Example user input
user_input = [[63, 1, 3, 145, 233,1,0,150,0,2,0,1,2,4]]  # Example parameters for prediction
recommended_doctors_indices = knn.kneighbors(user_input, return_distance=True)

In [18]:
recommended_doctors_indices

(array([[317.77836099, 317.85386563, 317.92736135, 317.93001525,
         317.93082595]]),
 array([[ 74, 464, 566, 487,  53]], dtype=int64))

In [43]:
# Load your data (example DataFrame: final_df)
# Normalize user health attributes
user_features = final_df[['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
                          'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']].values
scaler = MinMaxScaler()
user_features = scaler.fit_transform(user_features)

# Prepare doctor IDs and ratings
doctor_ids = final_df['doctor_id'].values
# Reshape ratings to a 2D array (required by the scaler)
# Encode doctor IDs to integers
label_encoder = LabelEncoder()
encoded_doctor_ids = label_encoder.fit_transform(doctor_ids)

# Convert encoded doctor IDs to one-hot encoding for classification
doctor_targets = to_categorical(encoded_doctor_ids)

# Split data into train and test sets
X_train_user, X_test_user, X_train_doctor, X_test_doctor, y_train, y_test = train_test_split(
    user_features, doctor_ids, doctor_targets, test_size=0.2, random_state=42)

# Define model inputs
user_input = Input(shape=(13,), name="User_Features")  # User health attributes
doctor_input = Input(shape=(1,), name="Doctor_ID")  # Doctor ID as input

# Embedding layer for doctor IDs
doctor_embedding = Embedding(input_dim=len(np.unique(doctor_ids)) + 1, 
                              output_dim=50, name="Doctor_Embedding")(doctor_input)
doctor_embedding = Flatten()(doctor_embedding)  # Flatten embedding layer

# Combine user features and doctor embedding
combined = Concatenate()([user_input, doctor_embedding])

# Dense layers
dense = Dense(128, activation='relu')(combined)
dense = Dense(64, activation='relu')(dense)
output = Dense(len(np.unique(doctor_ids)), activation='softmax')(dense)  # Softmax activation for multi-class

# Build and compile the model
model = Model(inputs=[user_input, doctor_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit([X_train_user, X_train_doctor], y_train, 
                    validation_data=([X_test_user, X_test_doctor], y_test),
                    epochs=10, batch_size=32)

loss, accuracy = model.evaluate([X_test_user, X_test_doctor], y_test)
print(f"Test Accuracy: {accuracy}")


Epoch 1/10


c:\Users\Chitt\Downloads\CSCE 5214\Heart Disease Prediction\venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['User_Features', 'Doctor_ID']. Received: the structure of inputs=('*', '*')
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0096 - loss: 5.0154 - val_accuracy: 0.0064 - val_loss: 5.0002
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0262 - loss: 4.9571 - val_accuracy: 0.0127 - val_loss: 4.9767
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0360 - loss: 4.8750 - val_accuracy: 0.0064 - val_loss: 4.9561
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0356 - loss: 4.7470 - val_accuracy: 0.0127 - val_loss: 4.9307
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0577 - loss: 4.5415 - val_accuracy: 0.0637 - val_loss: 4.8183
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2091 - loss: 4.2725 - val_accuracy: 0.1146 - val_loss: 4.5896
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3351 - loss: 3.8920 - val_accuracy: 0.1975 - val_loss: 4.2266
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4548 - loss: 3.3192 - val_accuracy: 0.4013 - val_loss: 3.7414
Ep

In [44]:
# Get predictions for the test set
doctor_predictions = model.predict([X_test_user, X_test_doctor])

# Convert y_test to single-label form
y_test_labels = y_test.argmax(axis=1)  # If y_test is one-hot encoded

# Convert predictions to single-label form
doctor_predictions_labels = doctor_predictions.argmax(axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test_labels, doctor_predictions_labels, average='weighted')
recall = recall_score(y_test_labels, doctor_predictions_labels, average='weighted')
f1 = f1_score(y_test_labels, doctor_predictions_labels, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Precision: 0.6273885350318471
Recall: 0.7006369426751592
F1 Score: 0.651501364877161


c:\Users\Chitt\Downloads\CSCE 5214\Heart Disease Prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Chitt\Downloads\CSCE 5214\Heart Disease Prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:
predicted_doctor_probs = model.predict([X_test_user, X_test_doctor])

# Convert probabilities to predicted class indices
predicted_doctor_indices = predicted_doctor_probs.argmax(axis=1)

# Map indices back to original doctor IDs
predicted_doctor_ids = label_encoder.inverse_transform(predicted_doctor_indices)
print(predicted_doctor_ids)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
[ 48   9 120  37 120 112  51  66  30 121 117  22  27  27 124 117 124   7
  94  24  49 113  60  52  57 119  47  23  92  48  95 130  14 118  16  22
  48  68  27   8  58   3  60 143  32 148   7 147  17  93  32  97   4  33
  94  32  32  99  60  25  91  61   6 119  34  23  35  94   6  94  89  66
 130  95 117 120  77  33  47  94  73  35  76  37  95 118  87 143 106   9
  91   9 117 136  52  35  23   9  38  92 143   7  24  30 135  16   4  32
  66 137 143 119  92  87  31  93  35  93  99  23 118  10  47  97  35  54
  73  80 121  24  32  94  84  89  99  23  74  83  80  31  17  14  73  99
  27 145  52  23   7 112  92  83  34 113  73  49  87]


In [60]:
# 1. Prepare the user's health parameters
# Let's say these are the user's health parameters
user_health_params = np.array([[63,1,3,145,233,1,0,150,0,2.3,0,0,1]])  # Example values

# Normalize the user's health parameters (same scaling as the training data)
user_health_params_scaled = scaler.transform(user_health_params)

# 2. Get doctor recommendations from the model
# Make a prediction for the user (this will give probabilities for each doctor)
doctor_probabilities = model.predict([user_health_params_scaled, np.zeros((user_health_params.shape[0], 1))])
print(len(doctor_probabilities[0]), "prob")
# 3. Sort the probabilities and select the top 5 doctors
# Get the indices of the top 5 doctors with the highest probabilities
top_5_doctors = np.argsort(doctor_probabilities[0])[::-1][:5]
print("top_5_doctors", top_5_doctors)
print(label_encoder.classes_)

# 4. Convert the indices back to doctor IDs
recommended_doctors = label_encoder.inverse_transform(top_5_doctors)

# Output the recommended doctors
print("Recommended Doctors:", recommended_doctors)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
150 prob
top_5_doctors [119   9  48  88  30]
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150]
Recommended Doctors: [120  10  49  89  31]


In [48]:
pickle.dump(model,open('doctor_recommend.pkl','wb'))

In [61]:
pickle.dump(label_encoder,open('doctor_label_encoder.pkl','wb'))